# Traveling Salesman Problem on Quantum Computer
The traveling salesman problem (TSP) is an NP-hard problem in combinatorial optimization. It attempts to find the shortest possible route when given a list of cities, and the distance between those cities, while only visiting each city once, and returning back to the starting city. The TSP attempts to be solved from an exact algorithm, however, once the problem size increases, the running time also increases. Heuristic algorithms were the next approach to limit running time, however these do not give the most optimal solution. That is where quantum algorithms come in. 

In order to run the Traveling Salesman on a Quantum Computer you must import the neccessary packages that utilize the graph plotting function.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx

## TSP using Qiskit on IBM Quantum
IBM has 21 quantum systems, 8 of which are open to the public. Each system has a dedicated number of qubits, or quantum bits, and a dedicated quantum volume. The highest of these being 65 qubits, and 32 quantum volume. They also have 5 simulators, which are classical emulators of quantum systems. The highest number of qubits of the simulators are 5000 qubits. In order to use IBM Quantum (IMBQ), you must implement Qiskit, a quantum package from IBM. IBMQ also has a Qiskit Textbook to learn the features of Qiskit and IBMQ.       

Solving the traveling salesman problem using quantum computing utilizes the built-in eigensolver function in Qiskit. First to use this function, you must create a graph with *n* nodes, then randomize the distance between the nodes. Then you must create the Hamiltonian circuit for the TSP of the graph. A hamiltonian circuit is a circuit that visits every vertex once, and it produces eigenvalues. Then using the built-in eigensolver function, from the hamiltonian, the smallest eigenvalue and eigenvector is determined. From there you can determine which qubit state is most likely with the result of the smallest eigenstate, and the order of which nodes are visited first. Then, you can determine the optimal distance. This method only works for a node of size *n=4* or lower. There is another method to run the TSP on a quantum simulator or quantum device. You must initialize the backend, or the quantum service, to do this you have to have an IBMQ account. You also have to use an optimizer to declare the amount of iterations. Then you define the circuit using the *TwoLocal* function, this is where you initialize the amount of qubits used and the types of gates for the quantum circuit. Then, using the *VQE* function to find the minimum eigenvalue of the hamiltonian, you can determine the qubit state that is most likely.

To begin, the neccessary Qiskit packages must be imported.

In [ ]:
from qiskit import IBMQ
from qiskit.circuit.library import TwoLocal
from qiskit.optimization.applications.ising import tsp
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua import QuantumInstance
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram

Next, a few definitions must be written to draw the tsp graph

In [ ]:
def draw_graph(G, colors, pos):
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)
    
def draw_tsp_solution(G, order, colors, pos):
    G2 = nx.DiGraph()
    G2.add_nodes_from(G)
    n = len(order)
    for i in range(n):
        j = (i + 1) % n
        G2.add_edge(order[i], order[j], weight=G[order[i]][order[j]]['weight'])
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(G2, node_color=colors, edge_color='b', node_size=600, alpha=.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G2, 'weight')
    nx.draw_networkx_edge_labels(G2, pos, font_color='b', edge_labels=edge_labels)

Next, a weight matrix is created in order to draw the tsp graph and create a distance matrix. 

In [ ]:
n = 4 #number of nodes
G = nx.Graph() #initializing graph G

# Computing the weight matrix
w = np.zeros([n,n])
for i in range(n):
    for j in range(n):
        temp = G.get_edge_data(i,j,default=0)
        if temp != 0:
            w[i,j] = temp['weight']

num_qubits = n**2 #number of qubits needed
ins = tsp.random_tsp(n, seed=123) #distance between each node
print('distance\n', ins.w) #printing the distance

# Draw the graph
G.add_nodes_from(np.arange(0, ins.dim, 1)) #adding each node to graph G
colors = ['r' for node in G.nodes()]

for i in range(0, ins.dim):
    for j in range(i+1, ins.dim):
        G.add_edge(i, j, weight=ins.w[i,j])    #adding the distance between each node to graph G

pos = {k: v for k, v in enumerate(ins.coord)} 

draw_graph(G, colors, pos) #drawing graph G

One way to solve the TSP is to use the built-in eigensolver function. First, create the Hamiltonian circuit for the graph, then, the smallest eigenvalue and eigenvector is determined. This produces which qubit state is most likely, which is then converted into the traveling salesman path. This method, however, only works for nodes sizes *n=4* and smaller.      

In [ ]:
qubitOp, offset = tsp.get_operator(ins) #creates the Hamiltonian for TSP of the graph
qp = QuadraticProgram()
qp.from_ising(qubitOp, offset, linear=True) #creates quadratic program

#Making the Hamiltonian in its full form and getting the lowest eigenvalue and eigenvector
eigenvalue = NumPyMinimumEigensolver(qubitOp) #determines the smallest eigenvalue and eigenvector
result = eigenvalue.run() 

print('tsp objective:', result.eigenvalue.real + offset) #prints the optimal eigenvalue
x = sample_most_likely(result.eigenstate) #determines which qubit state is most likely
print('feasible:', tsp.tsp_feasible(x)) #printing whether or not the solution is possible
z = tsp.get_tsp_solution(x) #determining which nodes are visited first
print('solution:', z) #printing the order of which nodes to visit first
print('solution objective:', tsp.tsp_value(z, ins.w)) #printing out the solution's optimal distance
draw_tsp_solution(G, z, colors, pos) #drawing the graph of the solution

Another method is to use IBM's quantum simulators or quantum devices. This method requires the initialization of the backend for where the TSP will run. To do this, an account is needed with IBMQ. The backend used in this example is "ibmq_qasm_simulator".

In [ ]:
#initializing the backend to run tsp on quantum simulator
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=10598, seed_transpiler=10598)

Then, using the *VQE* (variational-quantum eigensolver) function, the minimum eigenvalue of the hamiltonian circuit is found. The *VQE* function uses the *Two Local* function to create a circuit from the hamiltonian and the *SPSA* (Simultaneous Perturbation Stochastic Approximation) function is the optimizer that tells the *VQE* function how many times to run. This method also only works for nodes sizes *n=4* and smaller. The percent feasibility decreases with each increased node size and the amount of time the problem takes to solve increases.

In [ ]:
spsa = SPSA(maxiter=300)#the optimizer, states how many iterations
ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=7, entanglement='linear') #defines the circuit
vqe = VQE(qubitOp, ry, spsa, quantum_instance=quantum_instance)
#defines the VQE algorithm to run on quantum simulator

result = vqe.run(quantum_instance)#runs the circuit on quantum simulator

print('time:', result.optimizer_time)#prints the amount of time the problem takes to solve
x = sample_most_likely(result.eigenstate)#determines which qubit state is most likely
print('feasible:', tsp.tsp_feasible(x))#printing whether or not the solution is possible
z = tsp.get_tsp_solution(x)#determining which nodes are visited first
print('solution:', z)#printing the order of which nodes to visit first
print('solution objective:', tsp.tsp_value(z, ins.w))#printing out the solution's optimal distance
draw_tsp_solution(G, z, colors, pos)#drawing the graph of the solution

Even though both of these methods produce the optimal solution, they are not useful, due to the fact that the largest graph size is only *n=4*. 

## TSP using Leap and Ocean Software on a D'Wave Processor
Leap is a cloud-based quantum service that gives access to quantum computers and quantum-hybrid solvers. In order to get started with Leap, you must create an account. This account gives a trial plan with a limited amount of time to be used to submit problems onto the solvers. There are four solvers available through Leap, two hybrid solvers and two quantum solvers. One of the hybrid solvers uses binary quadratic modeling to solve optimization problems, whereas the other hybrid solver uses discrete quadratic modeling to solve optimization problems. The two quantum solvers are the Advantage and the DW-2000Q. The Advantage has 5,760 qubits, whereas the DW-2000Q has 2048 qubits. Both solvers support problem types of ising and qubo (quadratic unconstrained binary optimization). This explanation will not be using ising models. Leap also provides emulator solvers, which are quantum simulators, and VFYC (virtual full-yield chip) solvers, which emulate the DW-2000Q solver. These two solvers are not available with the Leap trial plan. In order to use the D'Wave solvers, you must implement the Ocean Software. D'Wave has system documentation to help get you started.


To begin, neccessary Ocean software must be imported.

In [ ]:
import dwave_networkx as dnx
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from dwave.system import LeapHybridSampler
import neal
import dimod
import hybrid 
import random 

First, a distance matrix is randomized to create the TSP graph. This example will be 10 nodes.

In [ ]:
G = nx.complete_graph(10)
for (u, v) in G.edges():
    G.edges[u,v]['weight'] = random.randint(0,10)

colors = ['r' for node in G.nodes()]
pos = nx.spring_layout(G)

draw_graph(G, colors, pos)

There are multiple ways of finding the most optimal path for the traveling salesman using D'Wave and Leap. The first one is using the traveling salesperson algorithm from an Ocean tool. It defines a ground state qubo (quadratic unconstrained binary optimization) that corresponds to the route with the smallest total edge weight. The algorithm requires a sampler to run the problem on. This sampler could be one of the hybrid solvers. The two most optimal samplers are a reference sampler called Kerberos, which runs on the Advantage solver, and Leap's Hybrid solver called LeapHybridSampler. This example provides both solvers to determine which is most optimal. 

In [ ]:
S = dnx.traveling_salesperson(G, sampler=hybrid.KerberosSampler()) #calculates TSP path using Kerberos solver
print(S)
cost = sum(G[n][nbr]["weight"] for n, nbr in nx.utils.pairwise(S)) #calculate total distance
print(cost)
draw_tsp_solution(G, S, colors, pos)

In [ ]:
S = dnx.traveling_salesperson(G, sampler=LeapHybridSampler()) #calculates TSP path using Leap Hybrid solver
print(S)
cost = sum(G[n][nbr]["weight"] for n, nbr in nx.utils.pairwise(S)) #calculate total distance
print(cost)
draw_tsp_solution(G, S, colors, pos)

Neither of these samplers is the most optimal, but they are very close to it. The other method starts by using the traveling salesperson algorithm to find the qubo, which will correspond to a minimum TSP route. Then it can be solved by using Kerberos and simply running the qubo on the Advantage solver. Then the qubit state with the lowest energy is returned and converted into the TSP path. Or, it can be solved using the LeapHybridSampler, which creates a binary quadratic model to run on Leap's Hybrid solver. The qubit with the lowest energy is also returned and then converted into the TSP path.

The difference between this method and the previous method is this method splits the algorithm up into miltiple algorithms, whereas the previous one solves it all in one algorithm. 

In order to convert the qubit state into a TSP path, a definition must be written to do that conversion.

In [ ]:
def binary_state_to_points_order(binary_state):
    """
    Transforms the the order of points from the binary representation: [1,0,0,0,1,0,0,0,1],
    to the binary one: [0, 1, 2]
    """
    points_order = []
    number_of_points = int(np.sqrt(len(binary_state)))
    for p in range(number_of_points):
        for j in range(number_of_points):
            if binary_state[(number_of_points) * p + j] == 1:
                points_order.append(j)
    return points_order

In [ ]:
Q= dnx.traveling_salesperson_qubo(G) #creating the qubo from the graph

In [ ]:
sampler = hybrid.KerberosSampler() #define which sampler is being used
result = sampler.sample_qubo(Q) #run the tsp on the Advantage
print(result.first.sample.values()) #print the qubit state with the lowest energy

In [ ]:
S= binary_state_to_points_order(#insert qubit state with lowest energy from previous output)
print(S)
cost = sum(G[n][nbr]["weight"] for n, nbr in nx.utils.pairwise(S))#calculate the total distance
print(cost)
draw_tsp_solution(G, S, colors, pos)

In [ ]:
bqm = dimod.BQM.from_qubo(Q) #create binary quadratic model from qubo
sampleset = LeapHybridSampler().sample(bqm) #run the tsp on Leap's Hybrid Processor
print(sampleset.first.sample.values()) #print the qubit state with the lowest energy

In [ ]:
S= binary_state_to_points_order(#insert qubit state with lowest energy from previous output)
print(S)
cost = sum(G[n][nbr]["weight"] for n, nbr in nx.utils.pairwise(S)) #calculate the total distance
print(cost)
draw_tsp_solution(G, S, colors, pos)

It is difficult to say which sampler is better at this point, however, using the traveling salesperson algorithm is much more optimal than the traveling salesperon qubo. Both methods work on much larger problem sizes and take a minimal amount of time. 

##### License and References

In [ ]:
import qiskit.tools.jupyter
%qiskit_copyright

https://github.com/networkx/networkx/blob/main/LICENSE.txt 

https://github.com/BOHRTECHNOLOGY/quantum_tsp/blob/master/LICENSE

https://docs.dwavesys.com/docs/latest/legal.html?highlight=license

"An investigation of IBM Quantum Computing device
performance on Combinatorial Optimisation Problems" 
url: https://arxiv.org/pdf/2107.03638.pdf
\bibitem[Khumalo \emph{et al.}(2021)]{2021arXiv210703638K}Khumalo, M.T., Chieza, H.A., Prag, K., and Woolway, M.: 2021, {\it arXiv e-prints}, arXiv:2107.03638.